<a href="https://colab.research.google.com/github/jnetoass/autoencoder/blob/main/autoencoder_dimensionalityreduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import plotly.graph_objects as go #criação de gráficos
from plotly.subplots import make_subplots  #criação de múltiplos grágicos
import plotly.express as px # criação de gráficos


[explicação das colunas](https://archive.ics.uci.edu/ml/datasets/heart+disease)

In [ ]:
dados = pd.read_csv('/content/heart-statlog_csv.csv')

In [ ]:
dados.head()

,age,sex,chest,resting_blood_pressure,serum_cholestoral,fasting_blood_sugar,resting_electrocardiographic_results,maximum_heart_rate_achieved,exercise_induced_angina,oldpeak,slope,number_of_major_vessels,thal,class
0,70,1,4,130,322,0,2,109,0,2.4,2,3,3,present
1,67,0,3,115,564,0,2,160,0,1.6,2,0,7,absent
2,57,1,2,124,261,0,0,141,0,0.3,1,0,7,present
3,64,1,4,128,263,0,0,105,1,0.2,2,1,7,absent
4,74,0,2,120,269,0,2,121,1,0.2,1,1,3,absent


In [ ]:
dados['class'] = [0 if x == 'absent' else 1 for x in dados['class']]

In [ ]:
X, y = dados.drop('class', axis=1), dados['class']

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(
     X, y, test_size=0.33, shuffle=True)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

def scale_datasets(x_train, x_test):
  """
  Standard Scale test and train data
  """
  standard_scaler = MinMaxScaler()
  x_train_scaled = pd.DataFrame(
      standard_scaler.fit_transform(x_train),
      columns=x_train.columns
  )
  x_test_scaled = pd.DataFrame(
      standard_scaler.transform(x_test),
      columns = x_test.columns
  )
  return x_train_scaled, x_test_scaled

In [ ]:
x_train_scaled, x_test_scaled = scale_datasets(x_train, x_test)

In [ ]:
x_train_scaled.shape, x_test_scaled.shape

((180, 13), (90, 13))

In [ ]:
stacked_encoder = tf.keras.Sequential([
                               tf.keras.layers.Dense(8, activation='LeakyReLU',input_shape=[13]),
                               tf.keras.layers.BatchNormalization(),
                               tf.keras.layers.Dense(4, activation='LeakyReLU'),
                               tf.keras.layers.BatchNormalization(),
                               tf.keras.layers.Dense(2, activation='linear')])
stacked_decoder = tf.keras.Sequential([
                               tf.keras.layers.Dense(4, activation='LeakyReLU',input_shape=[2]),
                               tf.keras.layers.BatchNormalization(),
                               tf.keras.layers.Dense(8, activation='LeakyReLU'),
                               tf.keras.layers.BatchNormalization(),
                               tf.keras.layers.Dense(13, activation='LeakyReLU')])

stacked_ae = tf.keras.Sequential([stacked_encoder,stacked_decoder])

stacked_ae.compile(loss=tf.keras.losses.MAE,
                optimizer=tf.keras.optimizers.Adam(learning_rate=0.05),
                metrics=['mae'])

In [ ]:
history = stacked_ae.fit(
    x_train_scaled, 
    x_train_scaled, 
    epochs=100, 
    batch_size=32 
    #validation_data=(x_test_scaled, x_test_scaled)
)

Epoch 1/100
6/6 [==============================] - 1s 4ms/step - loss: 0.4142 - mae: 0.4142
Epoch 2/100
6/6 [==============================] - 0s 5ms/step - loss: 0.2776 - mae: 0.2776
Epoch 3/100
6/6 [==============================] - 0s 4ms/step - loss: 0.2343 - mae: 0.2343
Epoch 4/100
6/6 [==============================] - 0s 5ms/step - loss: 0.2143 - mae: 0.2143
Epoch 5/100
6/6 [==============================] - 0s 4ms/step - loss: 0.1965 - mae: 0.1965
Epoch 6/100
6/6 [==============================] - 0s 5ms/step - loss: 0.1833 - mae: 0.1833
Epoch 7/100
6/6 [==============================] - 0s 7ms/step - loss: 0.1792 - mae: 0.1792
Epoch 8/100
6/6 [==============================] - 0s 5ms/step - loss: 0.1771 - mae: 0.1771
Epoch 9/100
6/6 [==============================] - 0s 4ms/step - loss: 0.1750 - mae: 0.1750
Epoch 10/100
6/6 [==============================] - 0s 4ms/step - loss: 0.1752 - mae: 0.1752
Epoch 11/100
6/6 [==============================] - 0s 5ms/step - loss: 0.1724 

In [ ]:
x_compressed = stacked_encoder.predict(x_test_scaled)

In [ ]:
x_compressed_train = stacked_encoder.predict(x_train_scaled)

In [ ]:
tf.random.set_seed(42)
model_2 = tf.keras.Sequential([
  tf.keras.layers.Dense(100,activation='LeakyReLU'),#,kernel_regularizer=tf.keras.regularizers.l2(0.01)),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Dense(100,activation='LeakyReLU'),#kernel_regularizer=tf.keras.regularizers.l2(0.01)),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Dense(10,activation='LeakyReLU'),#kernel_regularizer=tf.keras.regularizers.l2(0.01)),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Dense(1, activation='sigmoid')
])

model_2.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(learning_rate=0.005),
                metrics=['accuracy'])

Historia = model_2.fit(x_compressed_train,y_train, epochs=100, validation_split=0.1)
#Historia = model_2.fit(x_train,y_train, epochs=100, validation_split=0.1)

Epoch 1/100
6/6 [==============================] - 1s 48ms/step - loss: 0.6013 - accuracy: 0.6728 - val_loss: 0.4837 - val_accuracy: 0.7222
Epoch 2/100
6/6 [==============================] - 0s 9ms/step - loss: 0.4417 - accuracy: 0.8210 - val_loss: 0.5231 - val_accuracy: 0.7778
Epoch 3/100
6/6 [==============================] - 0s 9ms/step - loss: 0.4150 - accuracy: 0.8210 - val_loss: 0.4942 - val_accuracy: 0.8333
Epoch 4/100
6/6 [==============================] - 0s 8ms/step - loss: 0.4141 - accuracy: 0.8086 - val_loss: 0.4682 - val_accuracy: 0.8333
Epoch 5/100
6/6 [==============================] - 0s 9ms/step - loss: 0.4241 - accuracy: 0.8148 - val_loss: 0.4856 - val_accuracy: 0.8333
Epoch 6/100
6/6 [==============================] - 0s 8ms/step - loss: 0.4228 - accuracy: 0.8025 - val_loss: 0.4885 - val_accuracy: 0.8333
Epoch 7/100
6/6 [==============================] - 0s 9ms/step - loss: 0.4032 - accuracy: 0.8148 - val_loss: 0.4559 - val_accuracy: 0.8333
Epoch 8/100
6/6 [=========

In [ ]:
from sklearn.metrics import classification_report

y_preds = model_2.predict(x_compressed)
#y_preds = model_2.predict(x_test)


print(classification_report(y_test, tf.round(y_preds)))

              precision    recall  f1-score   support

           0       0.89      0.88      0.88        57
           1       0.79      0.82      0.81        33

    accuracy                           0.86        90
   macro avg       0.84      0.85      0.85        90
weighted avg       0.86      0.86      0.86        90



In [ ]:
from sklearn.metrics import confusion_matrix

y_preds = model_2.predict(x_compressed)
#y_preds = model_2.predict(x_test)


print(confusion_matrix(y_test, tf.round(y_preds)))

[[50  7]
 [ 6 27]]


In [ ]:
model_2.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 10)                50        
                                                                 
 dense_6 (Dense)             (None, 10)                110       
                                                                 
 dense_7 (Dense)             (None, 1)                 11        
                                                                 
Total params: 171
Trainable params: 171
Non-trainable params: 0
_________________________________________________________________


In [ ]:
tf.random.set_seed(42)
inputs = tf.keras.Input(x_compressed_train[1].shape)
x = tf.keras.layers.Dense(10, activation='relu')(inputs)
x = tf.keras.layers.Dense(10, activation='relu')(x)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)
functional_model = tf.keras.Model(inputs,outputs, name='functional_api')

functional_model.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(learning_rate=0.005),
                metrics=['accuracy'])

Historia2 = functional_model.fit(x_compressed_train,y_train, epochs=100, validation_split=0.1)

Epoch 1/100
6/6 [==============================] - 1s 55ms/step - loss: 0.8490 - accuracy: 0.4725 - val_loss: 0.7112 - val_accuracy: 0.4762
Epoch 2/100
6/6 [==============================] - 0s 11ms/step - loss: 0.7234 - accuracy: 0.4835 - val_loss: 0.6564 - val_accuracy: 0.4762
Epoch 3/100
6/6 [==============================] - 0s 13ms/step - loss: 0.6641 - accuracy: 0.5714 - val_loss: 0.6302 - val_accuracy: 0.6667
Epoch 4/100
6/6 [==============================] - 0s 12ms/step - loss: 0.6372 - accuracy: 0.6593 - val_loss: 0.6179 - val_accuracy: 0.7619
Epoch 5/100
6/6 [==============================] - 0s 11ms/step - loss: 0.6125 - accuracy: 0.7912 - val_loss: 0.6043 - val_accuracy: 0.8095
Epoch 6/100
6/6 [==============================] - 0s 10ms/step - loss: 0.5912 - accuracy: 0.7857 - val_loss: 0.5850 - val_accuracy: 0.8095
Epoch 7/100
6/6 [==============================] - 0s 10ms/step - loss: 0.5648 - accuracy: 0.7802 - val_loss: 0.5615 - val_accuracy: 0.8571
Epoch 8/100
6/6 [===

In [ ]:
y_preds = functional_model.predict(x_compressed)

print(classification_report(y_test, tf.round(y_preds)))

              precision    recall  f1-score   support

           0       0.76      0.76      0.76        42
           1       0.83      0.83      0.83        58

    accuracy                           0.80       100
   macro avg       0.79      0.79      0.79       100
weighted avg       0.80      0.80      0.80       100



In [ ]:
from sklearn.metrics import accuracy_score


accuracy_score(y_test, tf.round(y_preds))

# sem nada

In [ ]:
tf.random.set_seed(42)
model_2 = tf.keras.Sequential([
  tf.keras.layers.Dense(100,activation='LeakyReLU'),#,kernel_regularizer=tf.keras.regularizers.l2(0.01)),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Dense(100,activation='LeakyReLU'),#kernel_regularizer=tf.keras.regularizers.l2(0.01)),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Dense(10,activation='LeakyReLU'),#kernel_regularizer=tf.keras.regularizers.l2(0.01)),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Dense(1, activation='sigmoid')
])

model_2.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(learning_rate=0.005),
                metrics=['accuracy'])

#Historia = model_2.fit(x_compressed_train,y_train, epochs=100, validation_split=0.1)
Historia = model_2.fit(x_train,y_train, epochs=100, validation_split=0.1)

Epoch 1/100
6/6 [==============================] - 1s 45ms/step - loss: 0.6337 - accuracy: 0.6543 - val_loss: 3.4906 - val_accuracy: 0.5000
Epoch 2/100
6/6 [==============================] - 0s 8ms/step - loss: 0.5641 - accuracy: 0.6914 - val_loss: 0.7365 - val_accuracy: 0.7778
Epoch 3/100
6/6 [==============================] - 0s 8ms/step - loss: 0.5395 - accuracy: 0.7037 - val_loss: 0.8772 - val_accuracy: 0.7778
Epoch 4/100
6/6 [==============================] - 0s 7ms/step - loss: 0.4670 - accuracy: 0.7654 - val_loss: 0.5448 - val_accuracy: 0.7778
Epoch 5/100
6/6 [==============================] - 0s 8ms/step - loss: 0.4296 - accuracy: 0.8025 - val_loss: 3.1473 - val_accuracy: 0.5000
Epoch 6/100
6/6 [==============================] - 0s 8ms/step - loss: 0.4382 - accuracy: 0.7840 - val_loss: 0.7836 - val_accuracy: 0.7778
Epoch 7/100
6/6 [==============================] - 0s 8ms/step - loss: 0.3950 - accuracy: 0.7963 - val_loss: 2.7840 - val_accuracy: 0.5000
Epoch 8/100
6/6 [=========

In [ ]:
from sklearn.metrics import confusion_matrix

y_preds = model_2.predict(x_test)
#y_preds = model_2.predict(x_test)


print(confusion_matrix(y_test, tf.round(y_preds)))

[[46 11]
 [10 23]]


In [ ]:
y_preds = model_2.predict(x_test)

print(classification_report(y_test, tf.round(y_preds)))

              precision    recall  f1-score   support

           0       0.82      0.81      0.81        57
           1       0.68      0.70      0.69        33

    accuracy                           0.77        90
   macro avg       0.75      0.75      0.75        90
weighted avg       0.77      0.77      0.77        90

